In [203]:
import pandas as pd
import numpy as np

In [204]:
selloutData =pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [205]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [206]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"]<0 ,0, selloutData["QTY"])

In [207]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


### 53주차 제거

In [208]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

week분리

In [209]:
selloutData["WEEK"] = selloutData["YEARWEEK"]%100

year분리

##### 함수 만들기

In [210]:
def customFuntion(inputColumn):
    inputColumn =201601
    yearValue =inputColumn/100
    yearValue = int(yearValue)
    return yearValue

##### YEAR 컬럼 만들고 위 함수 적용하기

In [211]:
selloutData["YEAR"] =selloutData["YEARWEEK"].apply(customFuntion)

In [212]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2016
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2016
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2016
3,A60,PRODUCT47,201402,0.0,0.0,2,2016
4,A60,PRODUCT56,201402,23.0,23.0,2,2016


##### 1) round함수 /가 아니고 // 하면 소숫점 아래자리가 반올림되어 정수형으로 변화 ( 근데 round함수는 반올림을 하기 때문에 50주차 이상이되면 반올림이 되어 연수가 올라가기 때문에 적합하지 않은 방법임)

In [213]:
selloutData["YEAR"] = round(selloutData["YEARWEEK"]//100)

In [214]:
refinedData = selloutData[selloutData.WEEK < 53]

In [215]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2014
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2014
3,A60,PRODUCT47,201402,0.0,0.0,2,2014
4,A60,PRODUCT56,201402,23.0,23.0,2,2014


##### 2 )따라서 int형으로 형 변환을 해주는 게 적합한 방법임


In [216]:
##refinedData = selloutData[selloutData.YEARWEEK%100 < 53]

In [217]:
##refinedData["TEST"] = ((selloutData["YEARWEEK"]/100).astype(int))

In [218]:
##refinedData.head()

###  깰끔~ 해보이기 위해서 정렬!

In [219]:
sortKey = ["REGIONID","PRODUCT", "YEARWEEK"]

In [220]:
sortedData = refinedData.sort_values(sortKey)

In [221]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


##### 여기서 앞 줄 인덱스가 걸리적 거린다면 

In [222]:
step1Data =sortedData.reset_index(drop=True)

In [223]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,A00,PRODUCT34,201405,516.0,516.0,5,2014


### rolling 함수 사용하기!! (정렬을 하고 사용해야 함) 

1.기본 이동평균 함수

In [82]:
step1Data["MA"]=step1Data.NEW_QTY.rolling(window=15, center=True).mean()

In [83]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,NaN
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,NaN
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,NaN
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,NaN
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,NaN


#### 위에서 NAN을 없애려면 어떻게 해야 할까? 

##### min_period 이용--rolling함수:이동평균 구하는 함수, 
##### 해석 : (window=15:총 15개의 이동평균을 구하겠다, center=true: 내 자신을 7번째로, 즉 가운데로 놓고 이동평균을 구하겠다, min_periods =1 : 원래는 평균이 15개의 값을 구해야 하지만 최소로 1개 있을 때도 평균을 구하겠다, .mean()는 평균을 구하겠다-> .max(), .min()등으로 사용에 따라 변경 가능 )

In [84]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(
    window=15, center=True, min_periods =1).mean()

In [85]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


#### result라는 csv 파일로 ./ 현재 경로에 저장하는 명령어

In [86]:
step1Data.to_csv("./result.csv")

In [87]:
pwd

'C:\\Users\\SMART-12\\Python_Fintech\\4.04 Project Seasonality'

### merge

In [168]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [169]:
selectedData=step1Data.groupby(groupKey)[["NEW_QTY"]].mean()

윗줄의 new_qty를 이름만 qty_new로 바꾼것 (아랫줄)

In [196]:
selectedData.columns=["QTY_NEW"]

In [197]:
selectedData.head()

QTY_NEW
REGIONID PRODUCT   YEAR            
A00      PRODUCT34 2014  275.961538
                   2015   86.634615
                   2016   36.576923
         PRODUCT58 2014    2.673077
                   2015    5.711538

데이터 합치기

In [172]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


### ##selectedData를 다시 인덱스 형식으로 바꿔준다 (데이터를 조인 시키려면 인덱스 형식이어야 함)

In [199]:
spiltedData=selectedData.reset_index(drop=False)
spiltedData.head()

,REGIONID,PRODUCT,YEAR,QTY_NEW
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538


### 데이터 조인

In [174]:
mergeKey = ["REGIONID", "PRODUCT", "YEAR"]

In [233]:
##left_on과 right_on의 값이 같으면 on=mergeKey로 해도됨
mergedData= pd.merge(step1Data, spiltedData,\
                    left_on = mergeKey,right_on = mergeKey, \
                      how = "left")
mergedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,QTY_NEW
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,275.961538


In [176]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,QTY_NEW
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538
5,A00,PRODUCT34,201406,423.0,423.0,6,2014,480.230769,275.961538
6,A00,PRODUCT34,201407,407.0,407.0,7,2014,468.214286,275.961538
7,A00,PRODUCT34,201408,364.0,364.0,8,2014,456.666667,275.961538
8,A00,PRODUCT34,201409,470.0,470.0,9,2014,431.533333,275.961538
9,A00,PRODUCT34,201410,538.0,538.0,10,2014,402.066667,275.961538


데이터가 3배 늘어난다- mergeKey에서 요소 year을 빼니까 데이터 량이 늘어나는데, 총 3년이 있어서 3배가 늘어난다는 뜻의 아래 코드이다.

In [177]:
## mergeKey2 = ["REGIONID", "PRODUCT"]

In [178]:
## mergedData2 = pd.merge(step1Data, spiltedData,\
##                     left_on = mergeKey2, right_on = mergeKey2,\
##                     how = "left")
## mergedData2.head()

mergedData를 csv파일로 현재 경로에 저장

In [179]:
## mergedData.to_csv("./mergedData.csv")

## 그룹바이함수 만들기

In [243]:
groupKey = ["REGIONID","PRODUCT"]

In [244]:
def groupFunction(eachGroup):
    eachGroup["MA3"] =eachGroup["NEW_QTY"]

In [246]:
mergedData.groupby(groupKey).apply(groupFunction)
mergedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,QTY_NEW
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,275.961538


In [180]:
groupKey2 = ["REGIONID","PRODUCT"]

In [181]:
groupDataSet =mergedData.groupby(groupKey2)

In [182]:
def groupRolling(groupDataSet):
   
    #len(list(groupDataSet.groups))
    
    #oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[2])
    #oneGroup.head(1)

    indexGroupData = oneGroup.reset_index(drop=True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window=5,center=True).mean()
   
    return indexGroupData

In [183]:
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[2])

In [184]:
oneGroup.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,QTY_NEW
312,A00,PRODUCT59,201401,9988.0,9988.0,1,2014,6299.933333,27418.692308
313,A00,PRODUCT59,201402,9005.0,9005.0,2,2014,7296.333333,27418.692308
314,A00,PRODUCT59,201403,8416.0,8416.0,3,2014,8346.600000,27418.692308
315,A00,PRODUCT59,201404,8553.0,8553.0,4,2014,9418.600000,27418.692308
316,A00,PRODUCT59,201405,10355.0,10355.0,5,2014,10564.200000,27418.692308


In [185]:
finalResult = mergedData.groupby(groupKey2).apply(groupRolling)

In [234]:
finalResult.head()

REGIONID    PRODUCT  YEARWEEK      QTY  NEW_QTY  WEEK  \
REGIONID PRODUCT                                                             
A00      PRODUCT34 0      A00  PRODUCT59    201401   9988.0   9988.0     1   
                   1      A00  PRODUCT59    201402   9005.0   9005.0     2   
                   2      A00  PRODUCT59    201403   8416.0   8416.0     3   
                   3      A00  PRODUCT59    201404   8553.0   8553.0     4   
                   4      A00  PRODUCT59    201405  10355.0  10355.0     5   

                      YEAR            MA       QTY_NEW      MA2  
REGIONID PRODUCT                                                 
A00      PRODUCT34 0  2014   6299.933333  27418.692308      NaN  
                   1  2014   7296.333333  27418.692308      NaN  
                   2  2014   8346.600000  27418.692308   9263.4  
                   3  2014   9418.600000  27418.692308  10025.6  
                   4  2014  10564.200000  27418.692308  11683.0

groupdataset의 데이터 량 검색(groupkey2=regionid,product를 기준으로 묶은 그룹이 794개가 있다)

In [187]:
len(list(groupDataSet.groups))

794

groupDataSet아래 onegroup을 왜할까?  롤링함수를 써서 seasonality를 단순하게 구하고 싶기 때문이다.
모든 데이터를 롤링시키면 추세선이 이상해질 수 있다. 그러니까, 어떤 지역에 대한 추세선을 만들고 싶으면 해당 그룹만의 롤링을 구하는 것이 좋음(롤링이란 이동평균 구하는거)

첫번째 인덱스 [0]

In [188]:
oneGroup1 = groupDataSet.get_group(list(groupDataSet.groups)[0])
oneGroup1.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,QTY_NEW
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538


In [189]:
indexGroupData = oneGroup.reset_index(drop=True)
indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window=5, center=True).mean()